##differentiate MMD

In [ ]:
# import sys
# sys.path
#!pip install protobuf==3.20.*
!pip install numpy --upgrade

In [ ]:
!pip uninstall mxnet

In [ ]:
import numpy as np
import scipy.optimize
import tensorflow as tf
import numpy as np
from itertools import permutations

import matplotlib.pyplot as plt

# Constraints
def positivity(f):
    '''
    Constraint 1: 
    Ensures flow moves from source to target
    '''
    return f 

def fromSrc(f, wp, i, shape):
    """
    Constraint 2: 
    Limits supply for source according to weight
    """
    fr = np.reshape(f, shape)
    f_sumColi = np.sum(fr[i,:])
    return wp[i] - f_sumColi

def toTgt(f, wq, j, shape):
    """
    Constraint 3: 
    Limits demand for target according to weight
    """
    fr = np.reshape(f, shape)
    f_sumRowj = np.sum(fr[:,j])
    return wq[j] - f_sumRowj

def maximiseTotalFlow(f, wp, wq): 
    """
    Constraint 4: 
    Forces maximum supply to move from source to target
    """
    return f.sum() - np.minimum(wp.sum(), wq.sum())

# Objective function
def flow(f, D):
    """
    The objective function
    The flow represents the amount of goods to be moved 
    from source to target
    """
    f = np.reshape(f, D.shape)
    return (f * D).sum()

# Distance
def groundDistance(x1, x2, norm = 2):
    """
    L-norm distance
    Default norm = 2
    """
    return np.linalg.norm(x1-x2, norm)

# Distance matrix
def getDistMatrix(s1, s2, norm = 2):
    """
    Computes the distance matrix between the source
    and target distributions.
    The ground distance is using the L-norm (default L2 norm)
    """
    # Slow method
    # rows = s1 feature length
    # cols = s2 feature length
    numFeats1 = s1.shape[0]
    numFeats2 = s2.shape[0]
    distMatrix = np.zeros((numFeats1, numFeats2))

    for i in range(0, numFeats1):
        for j in range(0, numFeats2):
            distMatrix[i,j] = groundDistance(s1[i], s2[j], norm)

    # Fast method (requires scipy.spatial)
    #import scipy.spatial
    #distMatrix = scipy.spatial.distance.cdist(s1, s2)

    return distMatrix

# Flow matrix
def getFlowMatrix(P, Q, D):
    """
    Computes the flow matrix between P and Q
    """
    numFeats1 = P[0].shape[0]
    numFeats2 = Q[0].shape[0]
    shape = (numFeats1, numFeats2)

    # Constraints  
    cons1 = [{'type':'ineq', 'fun' : positivity},
             {'type':'eq', 'fun' : maximiseTotalFlow, 'args': (P[1], Q[1],)}]

    cons2 = [{'type':'ineq', 'fun' : fromSrc, 'args': (P[1], i, shape,)} for i in range(numFeats1)]
    cons3 = [{'type':'ineq', 'fun' : toTgt, 'args': (Q[1], j, shape,)} for j in range(numFeats2)]

    cons = cons1 + cons2 + cons3

    # Solve for F (solve transportation problem) 
    F_guess = np.zeros(D.shape)
    F = scipy.optimize.minimize(flow, F_guess, args=(D,), constraints=cons)
    F = np.reshape(F.x, (numFeats1,numFeats2))

    return F

# Normalised EMD
def EMD(F, D):  
    """
    EMD formula, normalised by the flow
    """
    return (F * D).sum() / F.sum()

# Runs EMD program  
def getEMD(P,Q, norm = 2):
    """
    EMD computes the Earth Mover's Distance between
    the distributions P and Q

    P and Q are of shape (2,N)

    Where the first row are the set of N features
    The second row are the corresponding set of N weights

    The norm defines the L-norm for the ground distance
    Default is the Euclidean norm (norm = 2)
    """  

    D = getDistMatrix(P[0], Q[0], norm)
    F = getFlowMatrix(P, Q, D)

    return EMD(F, D)
@tf.autograph.experimental.do_not_convert
def get_loss(pointclouds1, pointclouds2):
    loss = tf.numpy_function(getEMD, [pointclouds1,pointclouds2], tf.float64)
    return loss

In [ ]:


def sample_integers(n, shape):
    sample = tf.random_uniform(shape, minval=0, maxval=tf.cast(n, 'float32'))
    sample = tf.cast(sample, 'int32')
    return sample

def resample_rows_per_column(x):
    """Permute all rows for each column independently."""
    n_batch = tf.shape(x)[0]
    n_dim = tf.shape(x)[1]
    row_indices = sample_integers(n_batch, (n_batch * n_dim,))
    col_indices = tf.tile(tf.range(n_dim), [n_batch])
    indices = tf.transpose(tf.stack([row_indices, col_indices]))
    x_perm = tf.gather_nd(x, indices)
    x_perm = tf.reshape(x_perm, (n_batch, n_dim))
    return x_perm

def z_score(x):
    """
    Z_scores each dimension of the data (across axis 0)
    """
    #mean_vals = tf.reduce_mean(x,axis=0,keep_dims=True)
    #std_vals = tf.sqrt(tf.reduce_var(x,axis=0,keep_dims=True))
    mean_vals,var_vals = tf.nn.moments(x,axes=[0],keep_dims=True)
    std_vals = tf.sqrt(var_vals)
    x_normalized = (x - mean_vals)/std_vals
    return x_normalized

def cost_matrix(x,y,p=2):
    "Returns the cost matrix C_{ij}=|x_i - y_j|^p"
    x_col = tf.expand_dims(x,1)
    y_lin = tf.expand_dims(y,0)
    c = tf.reduce_sum((tf.abs(x_col-y_lin))**p,axis=2)
    return c

def asymmetric_loss(alpha):
    def loss(y_true, y_pred):
        delta = y_pred - y_true
        return K.mean(K.square(delta) * 
                      K.square(K.sign(delta) + alpha), 
                      axis=-1)
    return loss

def outer_sinkhorn_loss(n,niter,epsilon, p=2):
    def sinkhorn_loss(x,y):
        """
        Given two emprical measures with n points each with locations x and y
        outputs an approximation of the OT cost with regularization parameter epsilon
        niter is the max. number of steps in sinkhorn loop

        Inputs:
            x,y:  The input sets representing the empirical measures.  Each are a tensor of shape (n,D)
            epsilon:  The entropy weighting factor in the sinkhorn distance, epsilon -> 0 gets closer to the true wasserstein distance
            n:  The number of support points in the empirical measures
            niter:  The number of iterations in the sinkhorn algorithm, more iterations yields a more accurate estimate
        Outputs:

        """
        # The Sinkhorn algorithm takes as input three variables :
        C = cost_matrix(x, y,p=p)  # Wasserstein cost function

        # both marginals are fixed with equal weights
        mu = tf.constant(1.0/n,shape=[n])
        nu = tf.constant(1.0/n,shape=[n])
        # Elementary operations
        def M(u,v):
            "Modified cost for logarithmic updates"
            "$M_{ij} = (-c_{ij} + u_i + v_j) / \epsilon$"
            return (-C + tf.expand_dims(u,1) + tf.expand_dims(v,0) )/epsilon
        def lse(A):
            return tf.reduce_logsumexp(A,axis=1,keepdims=True)

        # Actual Sinkhorn loop
        u, v = 0. * mu, 0. * nu
        for i in range(niter):
            u = epsilon * (tf.math.log(mu) - tf.squeeze(lse(M(u, v)) )  ) + u
            v = epsilon * (tf.math.log(nu) - tf.squeeze( lse(tf.transpose(M(u, v))) ) ) + v

        u_final,v_final = u,v
        pi = tf.exp(M(u_final,v_final))
        cost = tf.reduce_sum(pi*C)
        return cost
    return sinkhorn_loss
    
def sinkhorn_from_product(x,epsilon,n,niter,z_score=False):
    y = resample_rows_per_column(x)
    if z_score:
        x = z_score(x)
        y = z_score(y)
    return sinkhorn_loss(x,y,epsilon,n,niter)

In [ ]:
import wishbone
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import keras
import keras.backend as K

import tensorflow as tf
#from keras.optimizers import adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import Input, Model
from tensorflow.keras import regularizers
from tensorflow.keras.losses import KLDivergence

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.stats import wasserstein_distance

#### define custom loss function

In [ ]:
def gaussian_kernel(x1, x2, beta = 1.0):
    r = tf.transpose(x1)
    r = tf.expand_dims(r, 2)
    return tf.reduce_sum(K.exp( -beta * K.square(r - x2)), axis=-1)
def MMD(x1, x2, beta):
    x1x1 = gaussian_kernel(x1, x1, beta)
    x1x2 = gaussian_kernel(x1, x2, beta)
    x2x2 = gaussian_kernel(x2, x2, beta)
    diff = tf.reduce_mean(x1x1) - 2 * tf.reduce_mean(x1x2) + tf.reduce_mean(x2x2)
    return diff


@tf.autograph.experimental.do_not_convert
def emd_loss(y_true, y_pred):
    y_true = y_true[:cells_to_use, ]
    return tf.py_function(wasserstein_distance, [y_true, y_pred], tf.float32)
def compute_cosine_distances(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance







@tf.autograph.experimental.do_not_convert
def val_loss_mmd2(x1, x2):
    x1 = tf.transpose(x1)
    x2 = tf.transpose(x2)
    res= MMD(x2, x1, 1)
    return res
def KL_div(P, Q):
    # First convert to np array
    P = np.array(P)
    Q = np.array(Q)
    #P = np.transpose(P)
    #Q = np.transpose(Q)
    #print(P.shape)
    #P, Q = Q, P
    # Then compute their means
    mu_P = np.mean(P, axis=0)
    mu_Q = np.mean(Q, axis=0)    
    
    # Compute their covariance
    cov_P = np.cov(P, rowvar=False)
    cov_Q = np.cov(Q, rowvar=False)    
        
    cov_Q_inv = np.linalg.inv(cov_Q)
    
    # Compute KL divergence
    KL_div = np.log(np.linalg.det(cov_Q)/np.linalg.det(cov_P)) - mu_P.shape[0] + np.trace(cov_Q_inv@cov_P) + \
                (mu_P - mu_Q).T@cov_Q_inv@(mu_P - mu_Q)
    
    KL_div = 0.5 * KL_div
    
    return KL_div/P.shape[0]
@tf.autograph.experimental.do_not_convert
def kl_in_tf(A, B):
    A = A[:cells_to_use, ]
    ans = tf.numpy_function(KL_div, [A, B], tf.float64)
    return ans

def mmd2(x1, x2):
    tf.shape(x1)
    x1 = tf.transpose(x1)
    x2 = tf.transpose(x2)
    res= MMD(x2, x1, 1)
    
@tf.autograph.experimental.do_not_convert
def mmd2(x1, x2):
    x1 = tf.transpose(x1)
    x2 = tf.transpose(x2)
    res= MMD(x2, x1, 1)
    return res
@tf.autograph.experimental.do_not_convert
def d_loss(x1, x2):
    ans = tf.reduce_mean(x2, axis = 0) - tf.reduce_mean(x1, axis = 0)
    ans = tf.reduce_mean(tf.square(ans))
    return tf.abs(ans)







@tf.autograph.experimental.do_not_convert
def mmd2_1(x1, x2):
    x1 = x1[:cells_to_use, ]
    x1 = tf.transpose(x1)
    x2 = tf.transpose(x2)
    res= MMD(x2, x1, 1)
    return res
@tf.autograph.experimental.do_not_convert
def d_loss_1(x1, x2):
    x1 = x1[:cells_to_use, ]
    ans = tf.reduce_mean(x2, axis = 0) - tf.reduce_mean(x1, axis = 0)
    ans = tf.reduce_mean(tf.square(ans))
    return tf.abs(ans)


@tf.autograph.experimental.do_not_convert
def distance_matrix(x1, x2):
    x1 = x1[cells_to_use:, ]
    ac = tf.expand_dims(x1, axis =0)
    c = tf.expand_dims(x1, axis =1)
    res_x1 = tf.norm(ac-c, axis= 2)
    #res_x1 = compute_cosine_distances(ac, c)
    ac = tf.expand_dims(x2, axis =0)
    c = tf.expand_dims(x2, axis =1)
    res_x2 = tf.norm(ac-c, axis= 2)
    #res_x2 = compute_cosine_distances(ac, c)
    ans = tf.reduce_mean(tf.square(res_x2-res_x1))
    return ans

@tf.autograph.experimental.do_not_convert
def wasserstein(x1, x2):
    x1 = x1[:cells_to_use, ]
    cdf_true = K.cumsum(x1, axis=0)
    cdf_pred = K.cumsum(x2, axis=0)
    emd = K.sqrt(K.mean(K.square(cdf_true - cdf_pred), axis=0))
    return K.mean(emd)


In [ ]:
@tf.autograph.experimental.do_not_convert
def mmd2_1(x1, x2):
    x1 = x1[:cells_to_use, ]
    x1 = tf.transpose(x1)
    x2 = tf.transpose(x2)
    #res= MMD(x2, x1, 1)
    res = maximum_mean_discrepancy(x1, x2)
    return res

In [ ]:
def maximum_mean_discrepancy(x, y):
    x_kernel = tf.reduce_mean(tf.exp(-tf.square(tf.math.subtract(x, tf.expand_dims(x, 1)))))
    y_kernel = tf.reduce_mean(tf.exp(-tf.square(tf.math.subtract(y, tf.expand_dims(y, 1)))))
    xy_kernel = tf.reduce_mean(tf.exp(-tf.square(tf.math.subtract(x, tf.expand_dims(y, 1)))))
    return x_kernel + y_kernel - 2 * xy_kernel

<h1>DEFINE MODELS</h1>

In [ ]:
def generator(x, y, size, steps_per_epoch):
    for i in range(steps_per_epoch):
        batch_x= x.sample(n = size, replace=True)
        batch_y= y.sample(n = size, replace=True)
        #yield (np.array(batch_x), np.concatenate((np.array(batch_y), np.array(batch_x)), axis =0))   ###why concat
        yield (np.array(batch_x), np.array(batch_y))

In [ ]:
@tf.autograph.experimental.do_not_convert
def model_that_corrects_batch_effect(unchanged_df_to_be_used_for_correction_from_batch,
                                     to_be_changed_batch_df,
                                     depth_of_the_model,
                                     epochs,
                                     steps_per_epoch,
                                     cells_per_batch,
                                     val_unchanged_df_to_be_used_for_correction_from_batch,
                                     val_to_be_changed_batch_df,
                                     first_activation,
                                     internal_activation
                                    ):
    l1_lambda = 0.01
    l2_lambda = 0.01
    
    input_shape = to_be_changed_batch_df.shape[1]
    inp = Input((input_shape,), name="input1")
    x = Dense(input_shape, activation=first_activation, kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(inp)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    n = input_shape
    for i in range(depth_of_the_model):
        n = n + 5
        x = Dense(n, activation= internal_activation, kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(x)
    output1 = Dense(input_shape, activation='relu', name="output11", kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(x)    
    adamoptimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
    new_model = Model(inputs=inp, outputs=[output1, output1])
    new_model.compile(
                     loss = [mmd2_1,
                              #wasserstein, 
                              d_loss_1, 
                              distance_matrix,
                              #KLDivergence()
                             ],
                     loss_weights= [0.50,
                                    0.30,
                                    0.20,
                                    #0.15, 
                                    #0.15
                                   ],
                
                      #loss= outer_sinkhorn_loss(n = cells_per_batch,niter = epochs, epsilon = 0.001, p=2), 
                       #loss = get_loss,
                      optimizer= adamoptimizer)
    es = EarlyStopping(monitor= "val_loss", mode='min', verbose=2, patience=10)
    if val_unchanged_df_to_be_used_for_correction_from_batch.empty:
                
        hist = new_model.fit_generator(generator(to_be_changed_batch_df, unchanged_df_to_be_used_for_correction_from_batch, 
                                       cells_per_batch, steps_per_epoch*epochs), 
                             steps_per_epoch = steps_per_epoch, 
                             epochs= epochs, 
                            )
    elif not val_unchanged_df_to_be_used_for_correction_from_batch.empty:
        hist = new_model.fit_generator(generator(to_be_changed_batch_df, unchanged_df_to_be_used_for_correction_from_batch, cells_per_batch, steps_per_epoch*epochs), 
                             validation_data = generator(val_to_be_changed_batch_df, val_unchanged_df_to_be_used_for_correction_from_batch, cells_per_batch, ((steps_per_epoch+5)*(epochs+5))),
                             validation_steps = steps_per_epoch,
                             steps_per_epoch = steps_per_epoch, 
                             epochs= epochs, 
                             
                             callbacks=[es]
                            )
        
    return([new_model, hist])

### model_2

In [ ]:
###generator
def generator2(x, size, steps_per_epoch):
    for i in range(steps_per_epoch):
        batch_x= x.sample(n = size, replace=True)
        yield np.array(batch_x)

In [ ]:
###model
def model_2():
    l1_lambda = 0.01
    l2_lambda = 0.01
    depth_of_the_model = 0
    input_shape = 22  ## hardcoded
    inp = Input((input_shape,), name="input1")
    x = Dense(input_shape, activation='linear', kernel_regularizer=regularizers.l1(l1_lambda))(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    n = input_shape
    for i in range(depth_of_the_model):
        n = n + 5
        x = Dense(n, activation='elu', kernel_regularizer=regularizers.l1(l1_lambda))(x)
    output1 = Dense(input_shape, activation='relu', name="output11", kernel_regularizer=regularizers.l1(l1_lambda))(x)    
    adamoptimizer = tf.keras.optimizers.Adam(learning_rate= 0.01)
    new_model = Model(inputs=inp, outputs=output1 )
    new_model.compile(
        loss = [mmd2, d_loss,],
        loss_weights= [0.50, 0.50,],
        #loss = mmd2, 
        optimizer= adamoptimizer)
    return new_model

### Data

In [ ]:
path = "Data/okokok"
list_files = os.listdir(path)

n1 = 6
new1 = list_files[n1]
print(new1)
new1 = path+"/" + new1
scdata1 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new1),
            cofactor= None)

n2= 8
new2 = list_files[n2]
print(new2)
new2 = path+"/" + new2
scdata2 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new2),
            cofactor= None)

new3 = list_files[n1+1]
print(new3)
new3 = path+"/" + new3
scdata3 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new3),
            cofactor= None)

new4 = list_files[n2+1]
print(new4)
new4 = path+"/" + new4
scdata4 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new4),
            cofactor= None)

In [ ]:
scaler1 = StandardScaler()
scaler1.fit(scdata1.data)

scaler5 = StandardScaler()
scaler5.fit(scdata3.data)

column_names = scdata1.data.columns

In [ ]:
scdata1.normalize()
scdata2.normalize()
scdata3.normalize()
scdata4.normalize()

In [ ]:
###data
source_dataset = scdata1.data
source_dataset1 = scdata2.data
target_dataset = scdata3.data
target_dataset2 = scdata4.data

In [ ]:
A = scdata1.data
B = scdata2.data
C = scdata3.data
D = scdata4.data

### get the base mmd value

In [ ]:
base_value_for_anchor = MMD(A, B, 1)
base_value_for_anchor.numpy()

In [ ]:
base_value_for_validation = MMD(C, D, 1)
base_value_for_validation.numpy()

### run model_2

In [ ]:
num_epochs = 30
number_of_cell =2000
source_iterator = generator2(source_dataset, number_of_cell, num_epochs)
source_iterator1 = generator2(source_dataset1, number_of_cell, num_epochs)
target_iterator = generator2(target_dataset, number_of_cell, num_epochs)
target_iterator1 = generator2(target_dataset, number_of_cell, num_epochs)
# Iterate over epochs
new_model = model_2()
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    source_batch = next(source_iterator)
    source_batch1 = next(source_iterator1)
    loss = new_model.train_on_batch(source_batch, source_batch1)  # Update the model with the source dataset
    print(f'Custom Loss: {loss}')
    
    target_batch = next(target_iterator)
    target_batch1 = next(target_iterator1)
    loss = new_model.train_on_batch(target_batch, target_batch1)  # Update the model with the target dataset
    print(f'Custom Loss: {loss}')



In [ ]:
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
number_of_cells_in_each_iteration = 500
first = f"Batch{N1}"    ###blue
second = f"Batch{N2}"   ###red

In [ ]:
A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)
B= scdata2.data.sample(n = number_of_cells_in_each_iteration, replace=True)
B = B.reset_index(drop=True)
AB_merged = A.append(B, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(new_model.predict(B))
res.columns = B.columns

A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)

AB_merged = A.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

validation

In [ ]:
C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = C.reset_index(drop=True)
D= scdata4.data.sample(n = number_of_cells_in_each_iteration, replace=True)
D = D.reset_index(drop=True)
CD_merged = C.append(D, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(new_model.predict(D))
res.columns = D.columns

C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = A.reset_index(drop=True)

CD_merged = C.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

###
###
###
###
###

### run model_1

In [ ]:
cells_to_use = 1500
model = model_that_corrects_batch_effect(A, B, 1, 10,5, cells_to_use, C, D,
                                         "linear",
                                         "linear"
#                                          pd.DataFrame(),
#                                          pd.DataFrame(),
                                          )

In [ ]:
hist= model[1]
model = model[0]

In [ ]:
model.summary()

In [ ]:
hist.history["loss"]

In [ ]:
#first_one = hist.history["loss"]
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
plt.plot(hist.history["loss"], c = "b", label = "training")
plt.plot([x for x in hist.history["val_loss"]], c = "r", label ="validation")
#plt.plot(first_one, c = "r", label ="validation")

plt.legend()
plt.title(f"Training and validation Losses \nof model for correcting batch effect \n between batches {N1} and {N2}")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.show()


<h1><b>Plot pca before correction of two batches</b></h1>

In [ ]:
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
number_of_cells_in_each_iteration = 500
first = f"Batch{N1}"    ###blue
second = f"Batch{N2}"   ###red

In [ ]:
A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)
B= scdata2.data.sample(n = number_of_cells_in_each_iteration, replace=True)
B = B.reset_index(drop=True)
AB_merged = A.append(B, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(model.predict(B)[0])
res.columns = B.columns

A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)

AB_merged = A.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

validation

In [ ]:
C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = C.reset_index(drop=True)
D= scdata4.data.sample(n = number_of_cells_in_each_iteration, replace=True)
D = D.reset_index(drop=True)
CD_merged = C.append(D, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(model.predict(D)[0])
res.columns = D.columns

C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = A.reset_index(drop=True)

CD_merged = C.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

In [ ]:
#################

In [ ]:
A = scdata1.data
B = scdata2.data
C = scdata3.data
D = scdata4.data

In [ ]:
res = pd.DataFrame(model.predict(D)[0])
res.columns = D.columns

In [ ]:
res.shape

In [ ]:
res.to_csv("MLcorrectedbatch2trainingdata.csv")

In [ ]:
res_sampled = res.sample(n = 500, replace=True)
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
fig = res_sampled.hist(ax = ax)

In [ ]:
C_sampled = C.sample(n = 500, replace=True)
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
fig = C_sampled.hist(ax = ax)

In [ ]:
res.head()

In [ ]:
res = res.astype('float64')

In [ ]:
res2 = tf.convert_to_tensor(res)

In [ ]:
result = pd.DataFrame(model.predict(B))

In [ ]:
result.head()

In [ ]:
mmd2(res, A)

In [ ]:
res.min()

In [ ]:
res2 = tf.convert_to_tensor(res)

In [ ]:
tf.reduce_sum(res2, axis = 0)

In [ ]:
K.mean(K.square(res.max() - A.max()))

In [ ]:
res = res.astype('float64')

In [ ]:
jj = mmd2(A, res)

In [ ]:
jj.numpy()

In [ ]:
j = B.iloc[1:5]
j

In [ ]:
res1 = correct_df_with_a_model(j, model)
res1

In [ ]:
model.predict(B.iloc[3].values.reshape(1, 22))

In [ ]:
AB_merged.tail(10)

In [ ]:
a = np.array([[1,2,3], [0, 0, 2], [1,2,3], [1, 2, 2]])
b = np.array([[1,2,3], [1, 2, 2], [0, 0, 2]])

In [ ]:
a= tf.convert_to_tensor(a)
b= tf.convert_to_tensor(b)
a = tf.cast(a, 'float64')
b = tf.cast(b, 'float64')

In [ ]:
tf.reduce_mean(a, axis = 0)

In [ ]:
tf.expand_dims(tf.transpose(a), axis =0)
tf.expand_dims(tf.transpose(a), axis =1)

In [ ]:
tf.expand_dims(tf.transpose(a), axis =1)

In [ ]:
y

In [ ]:
distance_matrix(a, b)

In [ ]:
first, second  = a.shape

In [ ]:
if (first > second):
    first, second = second, first

In [ ]:
first

In [ ]:
c = tf.reshape(a, [1, first, second])

In [ ]:
ac = tf.reshape(a, [first,1,second])

In [ ]:
c.shape

In [ ]:
z = tf.norm(ac-c, axis= 2)

In [ ]:
tf.reduce_mean(tf.square(z-z))

In [ ]:
K.mean(tf.norm(a-b, axis =1))

In [ ]:
def rowwise_diff(a, b):
    length = a.shape[0]
    result = sum([np.linalg.norm(a[i] - b[i]) for i in range(length) ])/length
    result = tf.convert_to_tensor(result)
    return result

In [ ]:
tf.convert_to_tensor(rowwise_diff(a, b))

In [ ]:
K.mean([np.linalg.norm(a[i] - b[i]) for i in range(a.shape[0]) ])